In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/17

%load_ext lab_black

In [2]:
import re
import sys

from collections import defaultdict
from itertools import cycle
from utils import read_input

In [3]:
def parse(direction):
    if direction == "<":
        return -1
    return 1


def get_pieces():
    pieces = []

    # |..@@@@.|
    # |.......|
    # |.......|
    # |.......|
    # +-------+
    pieces.append(
        {
            "id": 0,
            "top": lambda row: row,
            "coordinates": lambda row, col: (
                (row, col + 2),
                (row, col + 3),
                (row, col + 4),
                (row, col + 5),
            ),
        }
    )

    # |...@...|
    # |..@@@..|
    # |...@...|
    # |.......|
    # |.......|
    # |.......|
    # +-------+
    pieces.append(
        {
            "id": 1,
            "top": lambda row: row + 2,
            "coordinates": lambda row, col: (
                (row + 2, col + 3),
                (row + 1, col + 2),
                (row + 1, col + 3),
                (row + 1, col + 4),
                (row, col + 3),
            ),
        }
    )

    # |....@..|
    # |....@..|
    # |..@@@..|
    # |.......|
    # |.......|
    # |.......|
    # +-------+
    pieces.append(
        {
            "id": 2,
            "top": lambda row: row + 2,
            "coordinates": lambda row, col: (
                (row + 2, col + 4),
                (row + 1, col + 4),
                (row, col + 2),
                (row, col + 3),
                (row, col + 4),
            ),
        }
    )

    # |..@....|
    # |..@....|
    # |..@....|
    # |..@....|
    # |.......|
    # |.......|
    # |.......|
    # +-------+
    pieces.append(
        {
            "id": 3,
            "top": lambda row: row + 3,
            "coordinates": lambda row, col: (
                (row + 3, col + 2),
                (row + 2, col + 2),
                (row + 1, col + 2),
                (row, col + 2),
            ),
        }
    )

    # |..@@...|
    # |..@@...|
    # |.......|
    # |.......|
    # |.......|
    # +-------+
    pieces.append(
        {
            "id": 4,
            "top": lambda row: row + 1,
            "coordinates": lambda row, col: (
                (row + 1, col + 2),
                (row + 1, col + 3),
                (row, col + 2),
                (row, col + 3),
            ),
        }
    )

    return pieces

In [4]:
def can_move(tower, piece, row_offset, col_offset):
    for row, col in piece["coordinates"](row_offset, col_offset):
        if row <= 0:
            return False
        if col < 0 or col > 6:
            return False
        if col in tower.get(row, []):
            return False
    return True


def settle_piece(tower, piece, row_offset, col_offset):
    repeat = None
    for row, col in piece["coordinates"](row_offset, col_offset):
        if row not in tower:
            tower[row] = []
        tower[row].append(col)
    return tower, repeat


def find_repeat(data):
    if len(data) < 3:
        return
    return data[1], data[2]


def process_piece(
    tower, jets, piece, top, repeats, piece_number, jet_index, jet_length
):
    row = top + 4
    column = 0

    while True:
        direction = next(jets)
        jet_index = (jet_index + 1) % jet_length
        if can_move(tower, piece, row, column + direction):
            column += direction
        if can_move(tower, piece, row - 1, column):
            row -= 1
        else:
            tower, repeat = settle_piece(tower, piece, row, column)

            if piece["top"](row) > top:
                top = piece["top"](row)

            identifier = (piece["id"], jet_index)
            repeats[identifier].append({"row": row, "pieces": piece_number})
            repeat = find_repeat(repeats[identifier])

            return tower, top, repeats, repeat, jet_index


def get_remaining(count, first, second, tops, top, index):
    pieces = second["pieces"] - first["pieces"]
    cycles = count - index
    repeats = cycles // pieces
    index += repeats * pieces

    rows = second["row"] - first["row"]
    top += repeats * rows

    cycles_needed = count - index
    top += tops[first["pieces"] + cycles_needed] - tops[first["pieces"]]

    return top

In [5]:
def build_tower(jets, count_a, count_b):
    repeats = defaultdict(list)
    tops = defaultdict()
    top = 0
    tower = {}
    jet_length = len(jets)
    jets = cycle(jets)
    pieces = cycle(get_pieces())
    repeat_index = 0
    jet_index = 0
    answer_a = None

    for i in range(1, count_b + 1):
        tower, top, repeats, repeat, jet_index = process_piece(
            tower, jets, next(pieces), top, repeats, i, jet_index, jet_length
        )
        tops[i] = top

        if repeat:
            repeat_index = i
            break

        if i == count_a:
            answer_a = top

    if not answer_a:
        answer_a = get_remaining(count_a, *repeat, tops, top, repeat_index)

    answer_b = get_remaining(count_b, *repeat, tops, top, repeat_index)
    return answer_a, answer_b

In [6]:
jets = read_input(
    parent=__vsc_ipynb_file__, line_delimiter="", val_type=parse, sample="a"
)
answer_a, answer_b = build_tower(jets, 2022, 1000000000000)

print("part a:", answer_a)
print("part b:", answer_b)

part a: 3068
part b: 1514285714288


In [7]:
jets = read_input(parent=__vsc_ipynb_file__, line_delimiter="", val_type=parse)
answer_a, answer_b = build_tower(jets, 2022, 1000000000000)

print("part a:", answer_a)
print("part b:", answer_b)

part a: 3157
part b: 1581449275319
